# Previsões do Modelo Global x Previsões de Vendas Vibra

## Histórico de Atualizações

# Setup Inicial

## Imporação das bibliotecas

In [1]:
# Manipulação de dados
import numpy as np
import pandas as pd
import json

# Calculo de estatisticas
from scipy import stats

# Manipulação de datas
from time import strftime
from datetime import datetime
import pytz
from dateutil.relativedelta import relativedelta

# alertas e mensagens de execução
import logging
import warnings

# Visualização
import matplotlib.pyplot as plt
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.graph_objs import *

# Prophet para previsão de series de tempo
from prophet import Prophet


## Definições Gerais

In [2]:
# usado para remover as mensagens de output de processamento dos modelos prophet.
cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True

In [3]:
# definção dos diretórios relativos

# replace with your container name
blob_container_name = 'general' 

# replace with your relative folder path
blob_relative_path_raw = 'raw/mercado_potencial/'
blob_relative_path_enriched = 'enriched/mercado_potencial/'


# replace with your linked service name
linked_service_raw = 'LS_ADLS_RAW_01'
linked_service_enriched = 'LS_ADLS_ENRICHED_01'

ls_raw = mssparkutils.credentials.getPropertiesAll(linked_service_raw)
ls_enriched = mssparkutils.credentials.getPropertiesAll(linked_service_enriched)

converter_dic_raw = json.loads(ls_raw)
converter_dic_enriched = json.loads(ls_enriched)

end_point_raw = (converter_dic_raw['Endpoint'].split("/"))[2]
end_point_enriched = (converter_dic_enriched['Endpoint'].split("/"))[2]


dir_raw = 'abfss://%s@%s/%s' % (blob_container_name, end_point_raw, blob_relative_path_raw)
dir_enriched = 'abfss://%s@%s/%s' % (blob_container_name, end_point_enriched, blob_relative_path_enriched)


In [4]:
# # getting linked services connection string 
# adls_account_name = 'stedlk01dtandev'  
# sas_key = r"9999"

# dir_enriched = 'abfss://general@stedlk02dtandev.dfs.core.windows.net/enriched/mercado_potencial/'
# ls_enriched = 'LS_ADLS_ENRICHED_01'

## Funções Auxiliares

In [5]:
def modelo_final (dados, periodos = 12):
    r"""Ajusta modelo prophet de serie temporal.

        Define, ajusta e prevê modelo de serie temporal baseado na biblioteca prophet.
        Modelo com a possibilidade de inclusão de variáveis regressoras. 
        
    Parameters
    ----------
    dados: pd.DataFrame 
        Dataframe com variáveis exigidas para a criação do modelo prophet.
        No dataframe deve conter, obrigatoriamente, coluna 'ds': datatime da série e 
        'y': variável alvo do modelo.
        As variáveis regressoras, quando existire, devem estar presentes no mesmo dataframe.
    periodos: int
        Períodos, em meses, a serem previstos pelo modelo.
        Por padrão, doi feinido 12 meses a serem previstos.
    
    Returns
    -------
    previsao: pd.DataFrame
        Dataframe com os resultados do predict model adcionada das variáveis de input do modelo.
    modelo: prophet model
        Modelo prophet ajustado (fitted) 
    """

    regressores = dados.columns.to_list()[2:]       # os 2 primeiros são DataTime e o Y da serie temporal

    lst_results = ['ds','yhat','yhat_lower', 'yhat_upper',
                   'trend', 'trend_lower', 'trend_upper',
                   'yearly', 'yearly_lower', 'yearly_upper']
    
    modelo = Prophet(seasonality_mode='multiplicative',
                     yearly_seasonality=True, 
                     weekly_seasonality=False,
                     daily_seasonality=False,
                     changepoint_range=0.8,
                     changepoint_prior_scale=0.05)

    for i in range(len(regressores)):               # passeia por cada lista agregativa
        modelo.add_regressor(regressores[i], standardize=False)
                
    modelo.fit(dados)

    futuro = modelo.make_future_dataframe(periods=periodos, freq='MS') #MS início do mês 
    futuro = pd.merge(futuro, dados, on='ds', how='outer')
    futuro = futuro.fillna(method='ffill')
    
    previsao = modelo.predict(futuro)
    previsao = pd.merge(previsao[lst_results], dados, on='ds',how='outer') 

    return(previsao, modelo)

In [6]:
def prev_mkt(data, dados, lista_emp, lista_reg):
    """ Chamada de previsao dos modelos por empresa para o MKTShare

        Chamada de modelos e ajuste dos resultados previstos em único dataframe com coluna que diferencia as "Empresas"

    Parâmetros:
    ------------    
        data: pd.DataFrame
            Dataframe com as informações do mktshare ajusado
        lista_reg: lista de strings
            lista com o nome das coluna que serão ajustadas no modelo

    Retorno:
    -----------
        prev: pd.DataFrame
            Dataframe com as previsões associadas aos dados originais da dinamica e do MKTShare
    """
    prev = pd.DataFrame()
    for i in lista_emp:
        filtro = data['Empresa']==i
        df = data[filtro].iloc[:,0:2]           # remove a coluna Empresa
        df.reset_index(drop=True, inplace=True)
        df2 = pd.merge(df, dados, on='ds')      # agrega a dinamica
        previsao, mod = modelo_final(df2[lista_reg])
        previsao['Empresa'] = i                 # insere a coluna empresa no dataframee previsao
        prev = pd.concat([prev,previsao], ignore_index=True)
    return(prev)

In [7]:
def ajustar_data(df):
    """Cria data a partir de dados de ano e mês separados.

        Ajusta e cria coluna em dataframe a partir de informações de 
        ano e mês separadas e em formato string. 
        
    Parâmetros
    ----------
    df: pd.DdataFrame
        Dataframe contendo as colunas 'Ano de MESANO' e 'Num mês'
    
    Retorno
    -------
    df: pd.DdataFrame
        Dataframe contendo a coluna 'Data' em formato datatime
    """

    df['Data'] = pd.to_datetime(['-'.join(i) for i in zip(df['Ano de MESANO'].map(str),df['Num mês'].map(str))],format='%Y-%m')
    df.sort_values(by='Data', inplace=True)
    df = df.set_index('Data', drop=True)
    return (df)

In [8]:
def ajusta_mkt (df):
    """ Remove colunas não utilizadas

        Remove colunas desnecessárias e renomeia as utilizadas
    Parâmetros:
    -----------
    df: pd.DataFrame
        Dataframe original, com as culunas a serem removidas:
            - 'Ano de MESANO';
            - 'Num mês';
            -  'Mês de MESANO';
            - 'Nomes de medida'.
    
    Retorno:
    ----------
    df: pd.DataFrame
        Dataframe ajustado e com colunas renomeadas

    """
    df = df.drop(['Ano de MESANO', 'Num mês', 'Mês de MESANO','Nomes de medida'], axis=1)
    df.reset_index(drop=False, inplace=True)
    df.columns = ['ds','Empresa', 'y']   # já deixa o dataframe preparado para o parquet
    order = [0,2,1]
    df = df[df.columns[order]]
    return(df)

In [9]:
def agrega_global_mkt (df):
    """ Agraga Volume Global ao dataframe de MKT Share

        Agrega a coluna 'Empresa' o 'Global' que é o somatório de todas as outras empresas.
        Ou seja, recriando o Volume de Vendas para cada segmento conformo os dados da dinamica

    Parâmetros:
    -----------
    df: pd.DataFrame
        Dataframe original
    
    Retorno:
    ----------
    df: pd.DataFrame
        Dataframe ajustado com o Global incorporado as Empresas

    """

    df1 = pd.DataFrame(df.groupby(['ds']).sum().reset_index())
    df1['Empresa'] = 'Global'
    df2 = pd.concat([df, df1], axis=0, ignore_index=True)

    return(df2)

# Resgate dos Dados Dinâmica Global e MKT Share

## Dados da Dinâmica Global do Diesel e melhores regressoras para<br/> ajuste dos modelos Globais de TRR e Consumidor Final

In [10]:
dinamica   = pd.read_parquet(dir_enriched + 'volumetria/vol04_dinamica_diesel.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})
dinamica_par   = pd.read_parquet(dir_enriched + 'volumetria/vol04_dinamica_diesel_parcial.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})
cons_final = pd.read_parquet(dir_enriched + 'volumetria/vol04_metricas_global_ConsFinal.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})
trr = pd.read_parquet(dir_enriched + 'volumetria/vol04_metricas_global_TRR.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})

## Dados do MKT Share

Coleta dados e faz ajustes para poder compatibilizar os dados do mktshare aos da dinamica

In [11]:
arquivo = dir_raw + 'volumetria/Dados_MarketShare_CONS_TRR.xlsx'

mktcf = pd.read_excel(arquivo, sheet_name='CONSUMIDOR - Volumes',
                        storage_options = {'linked_service' : linked_service_raw})
mkttrr = pd.read_excel(arquivo, sheet_name='TRR - Volumes',
                        storage_options = {'linked_service' : linked_service_raw})

# arquivo = 'abfss://general@stedlk01dtandev.dfs.core.windows.net/raw/mercado_potencial/volumetria/Dados_MarketShare_CONS_TRR.xlsx'

# mktcf = pd.read_excel(arquivo, sheet_name='CONSUMIDOR - Volumes')
# mkttrr = pd.read_excel(arquivo, sheet_name='TRR - Volumes')

mktcf = ajustar_data(mktcf)
mkttrr = ajustar_data(mkttrr)

# alinhar data com as informações da dinamica
mktcf = mktcf['2012-06':]
mkttrr = mkttrr['2012-06':]

#Ajusta mkt para alinhar a dinamica
mktcf  = ajusta_mkt(mktcf)
mkttrr = ajusta_mkt(mkttrr)

# Inclui o Global as empresas
mktcf  = agrega_global_mkt(mktcf)
mkttrr = agrega_global_mkt(mkttrr)

## Verificando se existem diferenças entre os dados de volume da Dinamica e do MKTShare

Espera-se que os volumes de Consumidor Final e TRR da Dinâmica sejam compatíveis com os volumes Globais criados a partir das empresas no MKT Share.

In [12]:
# Dataframes auxiliares para coletar o MKTShare do Consumidor Final e do TRR
x1 = mktcf[mktcf['Empresa']=='Global'].reset_index(drop=True)
x2 = mkttrr[mkttrr['Empresa']=='Global'].reset_index(drop=True)

In [13]:
fig = make_subplots(rows=2, cols=1,
                    y_title='Volume (m³)',
                    subplot_titles=('Consumidor Final', 'TRR',),
                    shared_yaxes=False,
                    vertical_spacing=0.1 )

# grafico 01
fig.add_trace(go.Scatter(
            x = x1['ds'],
            y = x1['y'],
            line = dict(color='orange', width=5),
            name = 'MKT Share',
            legendgroup = 'MKT Share',
            showlegend=True),
    row=1, col=1)

fig.add_trace(go.Scatter(
        x=dinamica['ds'],
        y=dinamica['Volume_ConsumidorFinal'],
        line={'color': 'green'},
        name='Dinamica',
        legendgroup = 'Dinamica',
        showlegend=True),
    row=1, col=1)

# grafico 02
fig.add_trace(go.Scatter(
            x = x2['ds'],
            y = x2['y'],
            line = dict(color='orange', width=5),
            name = 'MKT Share',
            legendgroup = 'MKT Share',
            showlegend=False),
    row=2, col=1)

fig.add_trace(go.Scatter(
        x=dinamica['ds'],
        y=dinamica['Volume_TRR'],
        line={'color': 'green'},
        name='Dinamica',
        legendgroup = 'Dinamica',
        showlegend=False),
    row=2, col=1)

fig.update_xaxes(matches='x', rangeslider_visible=True, rangeslider_thickness = 0.05)
fig.update_xaxes(rangeslider= {'visible':False}, row=1, col=1)

fig.update_layout(
            title = 'Comparativo de Volumes' + '<br><sup>'+ 'Dinâmica Diesel x MKT Share' + '</sup>',
            #xaxis_title = "Período",
            #yaxis_title = "y_label",
            legend_title = "Legenda",
            font=dict(
                family = "Courier New, monospace",
                size = 14,
                color = "royalblue"),
            height=900
)

fig = plotly.offline.plot(fig, output_type='div')
#displayHTML(fig)
#fig.show()


# Ajuste dos Modelos para o MKT Share por 'Empresa'

## Lista de variáveis que serão ajustadas em seus respectivos modelos

In [14]:
# Consumidor Final
Best_Reg_CF = cons_final[cons_final.MAPE == cons_final.MAPE.min()].iloc[0,:][1].tolist()
lista_CF = ['ds','y']
lista_CF.extend(Best_Reg_CF)

# TRR
Best_Reg_TRR = trr[trr.MAPE == trr.MAPE.min()].iloc[0,:][1].tolist()
lista_TRR = ['ds','y']
lista_TRR.extend(Best_Reg_TRR)


In [15]:
print('Regressoras utilizadas no ajuste de modelo MKT Share Consumidor Final')
print(lista_CF)

print("")

print('Regressoras utilizadas no ajuste de modelo MKT Share TRR')
print(lista_TRR)

Os modelos de MKTShare foram ajustados con auxílio das regressoras dos respectivos segmentos definidas na exploração anterior

## Ajuste dos Modelos

In [16]:
empresas = ['VIBRA','RAIZEN', 'IPIRANGA','OUTRAS NP','Global']

# Previsão MKT Share Consumidor Final por empresa
prev_mktcf = prev_mkt(mktcf, dinamica_par, empresas, lista_CF)

# Previsão MKT Share TRR por empresa
prev_mkttrr = prev_mkt(mkttrr, dinamica_par, empresas, lista_TRR)

## Salvando as Previões para o MKT Share

In [17]:
# Salvando as previsoe s de MKT Share
prev_mktcf.to_parquet(dir_enriched + 'volumetria/vol06_prev_mkt_ConsFinal.parquet', 
                        storage_options = {'linked_service':linked_service_enriched})
prev_mkttrr.to_parquet(dir_enriched + 'volumetria/vol06_prev_mkt_TRR.parquet', 
                        storage_options = {'linked_service':linked_service_enriched})

# Visualização das Previsões

## Resgatando Dados das Previsões de Crescimento Vegetativo (Vendas Vibra)

In [18]:
prev_vibra  = pd.read_parquet(dir_enriched + 'volumetria/vol01_prev_vegetativo.parquet',
    storage_options = {'linked_service' : linked_service_enriched})

# Filtrando Consumidor Final e TRR
prev_vibracf  = prev_vibra[prev_vibra['Atv_Economica']=='CONSUMIDOR FINAL']
prev_vibratrr = prev_vibra[prev_vibra['Atv_Economica']=='TRR']

# remover o último mês  - normalmente mês corrente - dados imcompletos
prev_vibracf  = prev_vibracf[:-1] 
prev_vibratrr = prev_vibratrr[:-1]

## Ajustando dados da previsão do  MKT Share

In [19]:
prev_mktcf = prev_mktcf[prev_mktcf['ds']>='2016']
prev_mkttrr = prev_mkttrr[prev_mkttrr['ds']>='2016']

### Criando 'ygraf' variável gerada da união dos dados reais 'y' e dos dados previstos'yhat'


In [20]:
# Consumidor Final
prev_mktcf['ygraf'] = prev_mktcf['y']
prev_mktcf.loc[prev_mktcf['y'].isnull(),'ygraf'] = prev_mktcf['yhat']

# TRR
prev_mkttrr['ygraf'] = prev_mkttrr['y']
prev_mkttrr.loc[prev_mkttrr['y'].isnull(),'ygraf'] = prev_mkttrr['yhat']

In [21]:
def start_prev(df):
    # pegar ultima data real =  primeira da previsao
    pri = df.loc[pd.isna(df['y']), :].index[0] #primeiro nan
    start = df.loc[pri,'ds'] #- relativedelta(months=1) # inicio do periodo a ser projetado
    #start = start + relativedelta(months=3)
    #start = start.strftime('%Y-%m-%d')
    return(start)

## Representação Gráfica

In [22]:
# empresas = ['OUTRAS NP','IPIRANGA','RAIZEN','VIBRA']
# cores = ['gray','gold','purple','green']

empresas = ['VIBRA','RAIZEN', 'IPIRANGA','OUTRAS NP']
cores = ['green', 'purple', 'gold', 'gray']

pri = dinamica.loc[pd.isna(dinamica['PIB_Exportacao']), :] #primeiro nan da dinamica

inicio_prev = start_prev(prev_mktcf).strftime('%Y-%m-%d')
y1 = prev_mktcf['ygraf'].max()
y2 = prev_mkttrr['ygraf'].max()

fig = make_subplots(rows=2, cols=1,
                    y_title='Volume (m³)',
                    subplot_titles=('Consumidor Final', 'TRR',),
                    shared_yaxes=False,
                    vertical_spacing=0.1 )

# grafico 01
dt = prev_mktcf[prev_mktcf['Empresa']=='Global'].reset_index(drop=True) # Consumidor Final - MKT Share - Global(Somatorio das Empresa)
fig.add_trace(go.Scatter( 
            x = dt['ds'],
            y = dt['ygraf'],
            line = dict(color='red', width=4),
            name = 'MKTSH_Global',
            legendgroup = 'Global',
            showlegend=True),
    row=1, col=1)

fig.add_trace(go.Scatter( # Linha do Crescimento Vegetativo Vibra -  Consumidor Final
            x = prev_vibracf['ds'],
            y = prev_vibracf['yhat'],
            line = {'color': 'green', 'dash':'dot'},
            name = 'Veg_VIBRA',
            legendgroup = 'Veg_VIBRA',
            showlegend=True),
    row=1, col=1)

for c, i in enumerate(empresas):
# linhas do  Consumidor Final - MKT Share - por empresa
        dt = prev_mktcf[prev_mktcf['Empresa']==i].reset_index(drop=True)
        fig.add_trace(go.Scatter(
                        name = 'MKTSH_'+i,
                        legendgroup = 'MKTSH_'+i,
                        x = dt['ds'],
                        y = dt['ygraf'],
                        line = {'color': cores[c]},
                        visible=True,
                        stackgroup='one'),
                    row=1, col=1
        )

# grafico 02
dt = prev_mkttrr[prev_mkttrr['Empresa']=='Global'].reset_index(drop=True) # TRR - MKT Share - Global(Somatorio das Empresa)
fig.add_trace(go.Scatter(
            x = dt['ds'],
            y = dt['ygraf'],
            line = dict(color='red', width=4),
            name = 'MKTSH_Global',
            legendgroup = 'Global',
            showlegend=False),
    row=2, col=1)

fig.add_trace(go.Scatter( # Linha do Crescimento Vegetativo Vibra -  TRR
            x = prev_vibratrr['ds'],
            y = prev_vibratrr['yhat'],
            line = {'color': 'green', 'dash':'dot'},
            name = 'Veg_VIBRA',
            legendgroup = 'Veg_VIBRA',
            showlegend=False),
    row=2, col=1)

for c, i in enumerate(empresas):
# linhas do TRR
        dt = prev_mkttrr[prev_mkttrr['Empresa']==i].reset_index(drop=True)
        fig.add_trace(go.Scatter(
                        name = i,
                        legendgroup = 'MKTSH_'+i,
                        x = dt['ds'],
                        y = dt['ygraf'],
                        line = {'color': cores[c]},
                        visible=True,
                        stackgroup='one',
                        showlegend=False),
                    row=2, col=1
        )
    
fig.update_xaxes(matches='x', rangeslider_visible=True, rangeslider_thickness = 0.05)
fig.update_xaxes(rangeslider= {'visible':False}, row=1, col=1)

if len(pri)>0:
    fig.add_annotation(
            text= ('<b>Nota: </b> <br>De ' + pri.index[0].strftime('%b de %Y') +' a ' + start_prev(prev_mktcf).strftime('%b de %Y')) + 
            (' algumas regressoras tiveram dados imputados por meio de médias móveis.<br>Pois, estes dados ainda não foram disponibilizadas pelas suas fontes geradoras.'),
            showarrow=False,
            font=dict(size=12),
            align="left",
            x=-0.05,
            y=-0.24,
            xref='paper',
            yref='paper',
            xanchor='left',
            yanchor='bottom',
            xshift=-1,
            yshift=-5)

fig.add_annotation(
            showarrow=False,
            text='Gerado em ' + datetime.now().strftime("%d %b %Y às %H:%M:%S"),
            font=dict(size=12),
            align="right",
            x=1.18,
            y=-0.24,
            xref='paper',
            yref='paper',
            xanchor='right',
            yanchor='bottom',)

# Linhas de divisão das previsões
fig.update_layout(
    shapes=[
        dict(type="line", xref="x", yref="y",
                x0=inicio_prev, x1=inicio_prev, 
                y0=0, y1=y1*1.05,
                line=dict(color="black",
                          width=2,
                          dash="dot")
                ),
        dict(type="line", xref="x2", yref="y2",
                x0=inicio_prev, x1=inicio_prev, 
                y0=0, y1=y2*1.05,
                line=dict(color="black",
                          width=2,
                          dash="dot")
                ),
    ])

# Área de previsão
fig.add_vrect(x0=inicio_prev, x1=dt['ds'].max().strftime('%Y-%m-%d'), 
              annotation_text="Previsão",
              annotation_position="bottom left",  
              annotation_font_size=14,
              annotation_font_color="white",
              #fillcolor="gray",
              opacity=0.15,
              line_width=0)

# titulo
fig.update_layout(
                template='simple_white', # template ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]
                # plot_bgcolor='rgba(0, 0, 0, 0)',
                # paper_bgcolor='rgba(0, 0, 0, 0)',
                title = 'Volume de Diesel (m³)' + '<br><sup>'+ 
                        prev_mkttrr['ds'].min().strftime('%m-%Y')+' a ' + 
                        prev_mkttrr['ds'].max().strftime('%m-%Y')+'</sup>',
                #xaxis_title = "Período",
                #yaxis_title = "y_label",
                legend_title = "Legenda",
                font=dict(
                        family = "Courier New, monospace",
                        size = 14,
                        color = "royalblue"),
                height=900
)

fig = plotly.offline.plot(fig, output_type='div')
displayHTML(fig)
#fig.show()


# Acesso à Documentação

In [23]:
help(modelo_final)

In [24]:
help(prev_mkt)

In [25]:
help(ajustar_data)

In [26]:
help(ajusta_mkt)

In [27]:
help(agrega_global_mkt)